# Training The Proposed SimpleNet Model over the entire training set
## Description of Notebook
The notebook contains code used for training the proposed SimpleNet model. 
The model classifies preprocessed retinal OCT scans into three categories: urgent referral, routine referral
and normal. Since, the best performing model should be chosen among all trained model, we trained a total of 10 models and selected the best one.

## Description of SimpleNet Model
In the publication, a small CNN architecture with 30,793 parameters is used as the
classifier. The proposed SimpleNet consists of four convolutional layers, two
batch normalization layers, two max-pooling layers, and a final fully connected
output layer consisting of three nodes with softmax activation function. The training process uses Adam as the optimizer with a learning rate of 0.001,
while the “categorical cross-entropy” cost function is used to train the output
layer. The training was performed in batches of 128 images per step.

### Table of Contents
* [1. Importing all modules](#1)
* [2. Loading train and test data](#2)
* [3. Generator Classes using Sequence class](#3)
* [4. Functions for building and loading the transfer learning model](#4)
* [5. Functions for training and testing](#5)
* [6. Training the proposed SimpleNet model on entire training data](#6)
* [7. Performance metrics of proposed SimpleNet model on testing set](#7)

<a id="1"></a>
### 1. Importing all modules

In [1]:

import os
import keras
from keras.models import Sequential
from scipy.misc import imread
get_ipython().magic('matplotlib inline')
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense
import pandas as pd
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.utils import Sequence
from cv2 import * #Import functions from OpenCV
import cv2
import glob
from skimage.transform import resize
import tensorflow as tf
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from keras.models import model_from_json
import json
from statistics import mean
from sklearn.utils import shuffle
import time

Using TensorFlow backend.


<a id="2"></a>
### 2. Loading train and test data
Entire training data is loaded and shuffled randomly.

In [2]:
img = glob.glob("../input/no-med-filt-60/no_median_filter_60_200/no_median_filter_60_200/train/CNV/*.jpeg");
img = img + glob.glob("../input/no-med-filt-60/no_median_filter_60_200/no_median_filter_60_200/train/DME/*.jpeg");
l = len(img);
y = np.zeros((l,3))
y[:,2] =1
img = img + glob.glob("../input/no-med-filt-60/no_median_filter_60_200/no_median_filter_60_200/train/DRUSEN/*.jpeg");
m = len(img);
k = np.zeros((m-l,3));
k[:,1] = 1;
y = np.append(y,k, axis =0);
img = img + glob.glob("../input/no-med-filt-60/no_median_filter_60_200/no_median_filter_60_200/train/NORMAL/*.jpeg");
k = np.zeros((len(img)-m,3));
k[:,0] = 1;
y = np.append(y,k, axis =0);

img, y = shuffle(img,y)
img, y = shuffle(img,y)
img, y = shuffle(img,y)
img, y = shuffle(img,y)
img, y = shuffle(img,y)


t_img = []

for images in glob.glob("../input/no-med-filt-60/no_median_filter_60_200/no_median_filter_60_200/test/CNV/*.jpeg"):
    n = resize(imread(images,0), (60, 200,1))
    t_img.append(n)
    
for images in glob.glob("../input/no-med-filt-60/no_median_filter_60_200/no_median_filter_60_200/test/DME/*.jpeg"):
    n = resize(imread(images,0), (60, 200,1))
    t_img.append(n)
    
l = len(t_img);
t_y = np.zeros((l,3))
t_y[:,2] =1
  
for images in glob.glob("../input/no-med-filt-60/no_median_filter_60_200/no_median_filter_60_200/test/DRUSEN/*.jpeg"):
    n = resize(imread(images,0), (60, 200,1))
    t_img.append(n)    

     
m = len(t_img);
k = np.zeros((m-l,3));
k[:,1] = 1;
t_y = np.append(t_y,k, axis =0);
    
    
for images in glob.glob("../input/no-med-filt-60/no_median_filter_60_200/no_median_filter_60_200/test/NORMAL/*.jpeg"):
    n = resize(imread(images,0), (60, 200,1))
    t_img.append(n)
    
k = np.zeros((len(t_img)-m,3));
k[:,0] = 1;
t_y = np.append(t_y,k, axis =0);

t_img = np.asarray(t_img)
t_y = np.asarray(t_y)


<a id="3"></a>
### 3. Generator Classes using Sequence class
Since our training dataset is large, we would use the Sequence class to help us feed images in batches for training.


In [3]:
class My_Generator(Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size
        self.n = len(image_filenames)

    def __len__(self):
        return int(np.ceil(len(self.image_filenames) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx_min = idx*self.batch_size
        idx_max = np.amin([((idx+1)*self.batch_size),self.n])
        batch_x = self.image_filenames[idx_min:idx_max]
        batch_y = self.labels[idx_min:idx_max]
        X = np.array([
            resize(imread(file_name,0), (60, 200,1))
               for file_name in batch_x])
        y = np.array(batch_y)
        X, y = shuffle(X,y)
        return X,y


<a id="4"></a>
### 4. Functions for building and loading the transfer learning model

build_model() builds the architecture of proposed SimpleNet model whereas load_ith_model() 
loads the ith trial model from memory for testing. 

build_model_best() retrieves the model at its best performing epoch.

The proposed model has just **30,793** parameters. 

In [ ]:
def build_model():
    #load vgg16 without dense layer and with theano dim ordering
    model = Sequential()
        
    model.add(Conv2D(filters = 20, kernel_size = (5,5), input_shape = (60,200,1), activation = 'relu'))
    model.add(Conv2D(filters = 20, kernel_size = (5,5), activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (4, 4)))
    
    
    model.add(Conv2D(filters = 40, kernel_size = (3,3), activation = 'relu'))
    model.add(Conv2D(filters = 30, kernel_size = (3,3), activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (4, 4)))
    

    model.add(Flatten())
    model.add(Dense(units = 3, activation = 'softmax'))

    #compile the model
    model.compile(optimizer=Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def load_ith_model(i):
    json_file = open('model-'+str(i)+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights('model-'+str(i)+'.h5')

    #compile the model
    model.compile(optimizer=Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def build_model_best(i):
    json_file = open('bestmodel-'+str(i)+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights('bestmodel-'+str(i)+'.h5')

    #compile the model
    model.compile(optimizer=Adam(lr = 0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

<a id="5"></a>
### 5. Functions for training and testing
For each trial model i, train_save(i) trains the model, saves model-i and saves model-i at its best performing epoch. 

test_performance(i) loads the best_model of trial i and return the performance metrics of the model on the text set.

In [ ]:

def train_save(i):
    
    batch_size = 128

    trainGenerator = My_Generator(img,y,batch_size)
    valacc = 0.0
    for j in range(1,11):
        if j==1:
            model = build_model()
            if i==1 and j==1:
                model.summary()
        else:
            model = load_ith_model(i)
        history = model.fit_generator(
            generator = trainGenerator,
            steps_per_epoch=(len(img)//batch_size),
            epochs=1, verbose = 2, class_weight = [0.70596402, 4.19022748, 0.74357918], validation_data = (t_img,t_y))
        model_json = model.to_json()
        with open('model-'+str(i)+'.json', "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model.save_weights('model-'+str(i)+'.h5')
        
        if history.history['val_acc'][-1]>valacc:
            model_json = model.to_json()
            with open('bestmodel-'+str(i)+'.json', "w") as json_file:
                json_file.write(model_json)
            # serialize weights to HDF5
            model.save_weights('bestmodel-'+str(i)+'.h5')
            valacc = history.history['val_acc'][-1]
            
def test_performance(i):
    perf = {}
    
    for j in range(1,6):
        model = build_model_best(i)      
        ans = model.predict(t_img
        ans = np.argmax(ans, axis =1)
        perf[j]=performance_metrics(np.argmax(t_y, axis = 1),ans)
                            
    print("TEST ACCURACIES for Simple Net ##",i,"##")
    print_performance(perf)
    return perf[1]


**perfromance_metrics()** derives all performance measures from the confusion matrix whereas **print_performance()** prints the testing metrics in a readable format.

In [ ]:

def performance_metrics(t_y,ans):
    acc = accuracy_score(t_y,ans)
    target_names = ['CLass Normal', 'CLass Early', 'Class Late']
    cm = confusion_matrix(t_y, ans) 
    sens0 = cm[0,0]/(cm[0,0]+cm[0,1]+cm[0,2])
    spec0 = (cm[1,1]+cm[1,2]+cm[2,1] +cm[2,2])/((cm[1,1]+cm[1,2]+cm[2,1] +cm[2,2])+(cm[1,0]+cm[2,0]))

    sens1 = cm[1,1]/(cm[1,0]+cm[1,1]+cm[1,2])
    spec1 = (cm[0,0]+cm[2,0]+cm[0,2] +cm[2,2])/((cm[0,0]+cm[2,0]+cm[0,2] +cm[2,2])+(cm[0,1]+cm[2,1]))

    sens2 = cm[2,2]/(cm[2,0]+cm[2,1]+cm[2,2])
    spec2 = (cm[0,0]+cm[0,1]+cm[1,0] +cm[1,1])/((cm[0,0]+cm[0,1]+cm[1,0] +cm[1,1])+(cm[0,2]+cm[1,2]))
  
    rep=classification_report(t_y, ans, target_names=target_names, digits = 4, labels=range(0,3),output_dict=True)
    return [acc,sens0,spec0,sens1,spec1,sens2,spec2],cm,rep


def print_performance(p):
    a = list(p[1][2].keys())
    b = list(p[1][2][a[0]].keys())

    for i in range (0,1):
        s = np.zeros(7)
        pn = pe = pl = fn = fe = fl = rn = re = rl = 0
        for j in range(1,6):
            s = s + p[j][0]
            pn = pn +  p[j][2][a[0]][b[0]]
            pe = pe +  p[j][2][a[1]][b[0]]
            pl = pl +  p[j][2][a[2]][b[0]]
            fn = fn + p[j][2][a[0]][b[2]]
            fe = fe + p[j][2][a[1]][b[2]]
            fl = fl +  p[j][2][a[2]][b[2]]
            rn = rn +  p[j][2][a[0]][b[1]]
            re = re +  p[j][2][a[1]][b[1]]
            rl = rl +  p[j][2][a[2]][b[1]]

        print('\nAccuracy = ',round(s[0]*20,2))
        print("\n              sensitivity  specificity      precision      f1-score")
        print('\nClass Normal: ',round(s[1]*20,2), ",       ", round(s[2]*20,2), ",       ", round(pn*20,2),",       ", round(fn*20,2))
        print('\nClass Early:  ',round(s[3]*20,2), ",       ", round(s[4]*20,2), ",       ", round(pe*20,2),",       ", round(fe*20,2))
        print('\nClass Late:   ',round(s[5]*20,2), ",       ", round(s[6]*20,2), ",       ", round(pl*20,2),",       ", round(fl*20,2))

        print('\nAverage')
        print('\nsensitivity: ',round(mean([s[1],s[3],s[5]])*20,2), ", specificity: ",round(mean([s[2],s[4],s[6]])*20,2),
              ", precision: ", round(mean([pn,pe,pl])*20,2),", f1-score: ", round(mean([fn,fe,fl])*20,2))

 <a id="6"></a>
### 6. Training the proposed SimpleNet model on entire training data
This cell is not executed in this notebook as each training will yield different results. Kindly send me an email if you need the trained weights used in the publication.

In [4]:
each_perf ={}
for i in range(1,11):
    img, y = shuffle(img,y)
    img, y = shuffle(img,y)
    img, y = shuffle(img,y)
    train_save(i)
    each_perf[i] = test_performance(i)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 56, 196, 20)       520       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 192, 20)       10020     
_________________________________________________________________
dropout_1 (Dropout)          (None, 52, 192, 20)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 52, 192, 20)       80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 48, 20)        0         
_________________________________________________________________
conv2d_3 (Conv2D)    

<a id="7"></a>
### 7. Performance metrics of proposed SimpleNet Model on testing set

In [ ]:
def print_performance_average_each_model(p,i):
    a = list(p[i][2].keys())
    b = list(p[i][2][a[0]].keys())

    s = np.zeros(7)
    pn = 0
    pe = 0
    pl =0
    fn =0
    fe = 0
    fl = 0
    rn = 0
    re = 0
    rl = 0
    for j in range(i,i+1):
        s = s + p[j][0]
        pn = pn +  p[j][2][a[0]][b[0]]
        pe = pe +  p[j][2][a[1]][b[0]]
        pl = pl +  p[j][2][a[2]][b[0]]
        fn = fn + p[j][2][a[0]][b[2]]
        fe = fe + p[j][2][a[1]][b[2]]
        fl = fl +  p[j][2][a[2]][b[2]]
        rn = rn +  p[j][2][a[0]][b[1]]
        re = re +  p[j][2][a[1]][b[1]]
        rl = rl +  p[j][2][a[2]][b[1]]
    print("\n\n###### Metrics for Best Model (Trial ",i, ") ######" )
    print('\nacc = ',round(s[0]*100,2))
    print("              sensitivity  specificity      precision      f1-score")
    print('\nClass Normal: ',round(s[1]*100,2), ",       ", round(s[2]*100,2), ",       ", round(pn*100,2),",       ", round(fn*100,2))
    print('\nClass Early:  ',round(s[3]*100,2), ",       ", round(s[4]*100,2), ",       ", round(pe*100,2),",       ", round(fe*100,2))
    print('\nClass Late:   ',round(s[5]*100,2), ",       ", round(s[6]*100,2), ",       ", round(pl*100,2),",       ", round(fl*100,2))

    print('\n\n##### Average Performance Metrics for Best Model (Trial ', i, ') #####')
    avg_prec = round(p[j][2][a[5]][b[0]]*100,2)
    avg_sens = round(np.sum([s[1]*p[j][2][a[0]][b[3]],
                      s[3]*p[j][2][a[1]][b[3]],
                      s[5]*p[j][2][a[2]][b[3]]])*(100/p[j][2][a[5]][b[3]]),2)
    avg_spec = round(np.sum([s[2]*p[j][2][a[0]][b[3]],
                      s[4]*p[j][2][a[1]][b[3]],
                      s[6]*p[j][2][a[2]][b[3]]])*(100/p[j][2][a[5]][b[3]]),2)
    avg_f1 = round(p[j][2][a[5]][b[2]]*100,2)
    acc = round(s[0]*100,2)
    print('\nprecision: ',avg_prec,'sensitivity: ',avg_sens, ", specificity: ",avg_spec, ", f1-score: ", avg_f1)
    return [avg_prec, avg_sens, avg_spec, avg_f1, acc]


In [4]:
print_performance_average_each_model(each_perf,9)



###### Metrics for Best Model (Trial 9 ) ######

acc =  97.7
              sensitivity  specificity      precision      f1-score

Class Normal:  100.0 ,        99.07 ,        97.28 ,        98.62

Class Early:   95.2 ,        98.67 ,        95.97 ,        95.58

Class Late:    97.8 ,        98.8 ,        98.79 ,        98.29


##### Average Performance Metrics for Best Model (Trial 9 ) #####

precision:  97.7 sensitivity:  97.7 , specificity:  98.83 , f1-score:  97.7


[97.7, 97.7, 98.83, 97.7, 97.7]